In [1]:
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph


In [2]:
os.environ['GROQ_API_KEY'] = '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'

In [3]:
llm = ChatGroq(temperature=0, groq_api_key=os.getenv("GROQ_API_KEY"), model_name="llama-3.1-70b-versatile")

In [13]:
prompt_extract = PromptTemplate.from_template(
            """
            ###  your are interactive chatbot and answer the question 
            ### great and answer the question 
            """
        )
chain_extract = llm
# res = chain_extract.invoke(input={'hi hello could you give me this item for 100 rupe'})


In [5]:

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = chain_extract.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [6]:
config = {"configurable": {"thread_id": "abc123"}}

In [20]:

query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = llm.invoke(input_messages, config)
print(output.content)  # output contains all messages in state

Hi Bob, it's nice to meet you. Is there something I can help you with or would you like to chat?


In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    chain = prompt | llm
    response = chain.invoke(state)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

In [21]:
query = "Hi ! what is my name."

input_messages = [HumanMessage(query)]
output = llm.invoke(input_messages, config)
print(output.content)  # output contains all messages in state

I don't have any information about your name. This is the beginning of our conversation, and I don't have any prior knowledge about you. If you'd like to share your name, I'd be happy to chat with you.


In [1]:
import pandas as pd

# List of items with original and selling prices
items = [
    {"Item": "Smartphone", "Original Price": 500, "Selling Price": 550},
    {"Item": "Laptop", "Original Price": 800, "Selling Price": 880},
    {"Item": "Headphones", "Original Price": 50, "Selling Price": 60},
    {"Item": "Smartwatch", "Original Price": 150, "Selling Price": 165},
    {"Item": "Bluetooth Speaker", "Original Price": 100, "Selling Price": 120},
    {"Item": "Tablet", "Original Price": 300, "Selling Price": 350},
    {"Item": "Camera", "Original Price": 600, "Selling Price": 680},
    {"Item": "Gaming Console", "Original Price": 400, "Selling Price": 450},
    {"Item": "Wireless Mouse", "Original Price": 20, "Selling Price": 25},
    {"Item": "Keyboard", "Original Price": 30, "Selling Price": 40},
    {"Item": "Monitor", "Original Price": 200, "Selling Price": 230},
    {"Item": "Printer", "Original Price": 150, "Selling Price": 175},
    {"Item": "External Hard Drive", "Original Price": 80, "Selling Price": 95},
    {"Item": "Router", "Original Price": 70, "Selling Price": 85},
    {"Item": "Television", "Original Price": 900, "Selling Price": 1000},
    {"Item": "Air Conditioner", "Original Price": 1200, "Selling Price": 1300},
    {"Item": "Microwave Oven", "Original Price": 250, "Selling Price": 275},
    {"Item": "Refrigerator", "Original Price": 1000, "Selling Price": 1100},
    {"Item": "Washing Machine", "Original Price": 700, "Selling Price": 780},
    {"Item": "Vacuum Cleaner", "Original Price": 150, "Selling Price": 175}
]

# Create a pandas DataFrame from the list
df = pd.DataFrame(items)

# Output the DataFrame to a CSV file
df.to_csv("ecommerce_items.csv", index=False)

print("CSV file saved successfully!")


CSV file saved successfully!


In [9]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
import pandas as pd

df = pd.read_csv("ecom_items.csv")

engine = create_engine("sqlite:///product.db")
df.to_sql("product", engine, index=False)

19

In [10]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM product WHERE Original_Price > 400;"))

sqlite
['product']
[('Smartphone', 'TechInnovate', 500, 550, 'A sleek and powerful smartphone with a 6.5-inch display, 128GB storage, and a 48MP camera for stunning photography.'), ('Laptop', 'Computech', 800, 880, 'High-performance laptop with a 15.6-inch Full HD display, 16GB RAM, and Intel i7 processor for multitasking and gaming.'), ('Camera', 'SnapVision', 600, 680, '24.1MP DSLR camera with 4K video recording, optical zoom, and a large ISO range for professional-level photography.'), ('Television', 'CineView', 900, 1000, '55-inch 4K UHD Smart TV with HDR support, built-in streaming apps, and voice control for a hands-free experience.'), ('Air Conditioner', 'CoolBreeze', 1200, 1300, '1.5-ton split air conditioner with fast cooling, energy efficiency, and smart temperature control via mobile app.'), ('Refrigerator', 'ChillPro', 1000, 1100, 'Double-door refrigerator with frost-free cooling, energy-efficient inverter technology, and ample storage space.'), ('Washing Machine', 'WashMat

In [14]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [17]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

df = pd.read_csv("ecom_items.csv")
tool = PythonAstREPLTool(locals={"df": df})


In [19]:
llm_with_tools = llm.bind_tools([tool], tool_choice=tool.name)
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser

parser = JsonOutputKeyToolsParser(key_name=tool.name, first_tool_only=True)

In [ ]:
system = f"""You have access to a pandas dataframe `df`. 

Given a user question, give the answer according to question"""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])
code_chain = prompt | llm_with_tools | parser |tool
code_chain.invoke({"question": "What's is price of Refrigerator"})

ValueError: This agent relies on access to a python repl tool which can execute arbitrary code. This can be dangerous and requires a specially sandboxed environment to be safely used. Please read the security notice in the doc-string of this function. You must opt-in to use this functionality by setting allow_dangerous_code=True.For general security guidelines, please see: https://python.langchain.com/v0.2/docs/security/

In [33]:
from langchain_community.document_loaders.csv_loader import CSVLoader

file_path = (
    "ecom_items.csv"
)

loader = CSVLoader(file_path=file_path)
data = loader.load()

In [34]:
prompt = PromptTemplate.from_template(
     """From the {data} and the {query}, identify the product asked by the user and return the name , price  """ 
    ""
 
)
chain = prompt |llm
query ='what is price of wasing machine'
res = chain.invoke(input={'data':str(data),'query':str(query)})

In [35]:
res.content

'To identify the product and return the name and price, we can iterate through the list of documents and check if the item name matches the product asked by the user. Here\'s a Python code snippet that accomplishes this:\n\n```python\nclass Document:\n    def __init__(self, metadata, page_content):\n        self.metadata = metadata\n        self.page_content = page_content\n\n    def get_item_info(self):\n        lines = self.page_content.split(\'\\n\')\n        item_info = {}\n        for line in lines:\n            key, value = line.split(\': \')\n            item_info[key] = value\n        return item_info\n\n\ndef get_product_info(documents, product_name):\n    for document in documents:\n        item_info = document.get_item_info()\n        if item_info[\'Item\'] == product_name:\n            return item_info[\'Item\'], item_info[\'Selling_Price\']\n    return None, None\n\n\n# Example usage:\ndocuments = [\n    Document(metadata={\'source\': \'ecom_items.csv\', \'row\': 0}, page_

In [ ]:
loader = CSVLoader("./banklist.csv", encoding="windows-1252")
documents = loader.load()

db = Chroma.from_documents(documents, embedding_function)
query = "Did a bank fail in North Carolina?"
docs = db.similarity_search(query)
print(docs[0].page_content)

In [11]:
product_details = str(res.content)
product_dict = {}

# Split the string by lines
lines = product_details.split('\n')

# Parse each line to extract key-value pairs
for line in lines:
    key, value = line.split(': ', 1)
    product_dict[key] = value

# Output the resulting dictionary
print(product_dict['Item'])

Washing Machine
